In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy

from importlib import reload
from matplotlib import rc
from mpl_lego import style
from neurobiases import EMSolver, TriangularModel
from sklearn.decomposition import PCA
%matplotlib inline

In [ ]:
style.use_latex_style()

In [ ]:
K = 1

In [ ]:
# Create triangular model and draw data
tm = TriangularModel(
    model='linear',
    parameter_design='direct_response',
    M=10,
    N=10,
    K=K,
    corr_cluster=0.05,
    corr_back=0.,
    coupling_distribution='gaussian',
    coupling_sparsity=0.5,
    coupling_loc=0,
    coupling_scale=0.25,
    coupling_rng=2332,
    tuning_distribution='gaussian',
    tuning_sparsity=0.5,
    tuning_loc=0,
    tuning_scale=0.25,
    tuning_rng=23456542,
    stim_distribution='uniform')
X, Y, y = tm.generate_samples(n_samples=2000, rng=2332)

In [ ]:
# Run sparse solver
solver = EMSolver(
    X, Y, y, K=K,
    solver='scipy_lbfgs',
    max_iter=1000,
    tol=1e-7,
    penalize_B=False,
    rng=948512,
    fit_intercept=False,
    Psi_transform=None,
    initialization='random')

In [ ]:
n_deltas = 20
deltas_init = np.sort(np.insert(np.linspace(-1.1, 1.1, n_deltas), 0, 0))
n_deltas = deltas_init.size
init_params = np.zeros((n_deltas, solver.get_params().size))

for idx, delta in enumerate(deltas_init):
    copy = solver.copy()
    copy.identifiability_transform(delta)
    print(copy.Psi_tr_to_Psi()[0])
    init_params[idx] = copy.get_params(return_Psi=True)

In [ ]:
deltas_fit = np.linspace(-0.18, 0.18, n_deltas)
fit_ident_params = np.zeros((n_deltas, solver.get_params().size))

fit_solver = solver.copy().fit_em()

for idx, delta in enumerate(deltas_fit):
    copy = fit_solver.copy()
    copy.identifiability_transform(delta)
    print(copy.Psi_tr_to_Psi()[0])
    fit_ident_params[idx] = copy.get_params(return_Psi=True)

In [ ]:
fit_params = np.zeros((n_deltas, solver.get_params().size))

for idx, delta in enumerate(deltas_init):
    print(idx)
    solver_copy = solver.copy()
    solver_copy.identifiability_transform(delta)
    solver_copy.fit_em()
    fit_params[idx] = solver_copy.get_params(return_Psi=True)

In [ ]:
pca = PCA(2)
fit_transformed = pca.fit_transform(fit_ident_params)
fit_per_transformed = pca.transform(fit_params)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(
    fit_transformed[:, 0],
    fit_transformed[:, 1],
    color='black',
    lw=3,
    zorder=-1)
ax.scatter(
    fit_per_transformed[:, 0],
    fit_per_transformed[:, 1],
    marker='o',
    color='red',
    lw=3,
    s=100,
    edgecolor='white',
    linewidth=0.6,
    alpha=0.75)

ax.set_xlabel(r'\textbf{PC 1 (Linear subspace)}', fontsize=18)
ax.set_ylabel(r'\textbf{PC 2 (Private variance)}', fontsize=18)
ax.tick_params(labelsize=12)
plt.savefig('identifiability_mapping_experiment.pdf', bbox_inches='tight')